In [ ]:
%pip install llama-index

In [ ]:
%pip install langchain

In [ ]:
from llama_index import SimpleDirectoryReader, GPTListIndex, GPTVectorStoreIndex, LLMPredictor, PromptHelper, ServiceContext, StorageContext, load_index_from_storage
from langchain import OpenAI
import sys
import os 

In [ ]:
import openai
os.environ["OPENAI_API_KEY"] = "***************************************" 
openai.api_key = os.environ["OPENAI_API_KEY"]


In [ ]:
from llama_index import ServiceContext, load_index_from_storage
from llama_index.embeddings.openai import OpenAIEmbedding

def create_index(path):
    max_input = 4096
    tokens = 246
    chunk_size = 600 
    max_chunk_overlap = 0.1
    
    #define prompt
    promptHelper = PromptHelper(max_input,tokens,max_chunk_overlap,chunk_size_limit=chunk_size)
        
    #define LLM
    llmPredictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-ada-001",max_tokens=tokens))
    
    #load data
    docs = SimpleDirectoryReader(path).load_data()
    
    #create vector index
    service_context = ServiceContext.from_defaults(llm_predictor=llmPredictor, prompt_helper=promptHelper)
    
    #create vector index
    embed_model = OpenAIEmbedding(embed_batch_size=1)
    vectorIndex = GPTVectorStoreIndex.from_documents(documents=docs, service_context=service_context, embed_model=embed_model, )
    # print("VI1")
    vectorIndex.storage_context.persist(persist_dir='Store')
    # print("VI2")
    print(vectorIndex)
    
    
    return vectorIndex

In [ ]:
create_index("Knowledge")

In [ ]:
def answerMe(question):
    storage_context = StorageContext.from_defaults(persist_dir = 'Store')
    index = load_index_from_storage(storage_context)
    query_engine = index.as_query_engine()
    response = query_engine.query(question)
    return response

In [ ]:
print(answerMe("who is author of the book?"))

In [ ]:
print(answerMe("Why are dinosaurs extinct, while humans are able to live?"))

In [ ]:
print(answerMe("Did you find any theories that you believe are not completely true?"))